In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import os
import numpy as np
np.set_printoptions(precision=4)
import catboost
from sklearn.metrics import accuracy_score, recall_score, precision_score , confusion_matrix , classification_report
import matplotlib.pyplot as plt
import seaborn as sns


Importing Data

In [ ]:
train = pd.read_csv (r'/kaggle/input/credit-card-customers/BankChurners.csv')
print (train)
df = pd.read_csv(r'/kaggle/input/credit-card-customers/BankChurners.csv')
ts = pd.Series(np.random.randn(1000), index=pd.date_range("1/1/2000", periods=1000))


In [ ]:
sns.heatmap(df.isna())

In [ ]:
df.hist( bins=30, figsize=(15,15) );


In [ ]:
plt.figure(figsize=(15,7))
g= sns.countplot(x='Marital_Status', data=df);
for p in g.patches:
    height = p.get_height()
    g.text(p.get_x()+p.get_width()/2.,
            height/2,
            '{}'.format(height),
            ha="center", color='white')

Removing the stuff they asked to remove


In [ ]:
y = train.Attrition_Flag
X = train.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2','Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1','Attrition_Flag'], axis=1)

Converting the classifier values to numerical values


In [ ]:
X = pd.get_dummies(X, columns=[ 'Gender', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category'])

In [ ]:
X.isnull().sum()

In [ ]:
Xnew = X.drop(columns=[ 'Gender_F', 'Education_Level_College', 
                                 'Marital_Status_Divorced', 'Income_Category_Unknown', 'Card_Category_Blue'])

In [ ]:
Xnew.columns

Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=1234)

In [ ]:
#cat_features = list(range(0, X.shape[1]))
#print(cat_features)

In [ ]:
from catboost.utils import create_cd
feature_names = dict()
for column, name in enumerate(train):
    if column == 0:
        continue
    feature_names[column - 1] = name
    
create_cd(
    label=0, 
    cat_features=list(range(1, train.columns.shape[0])),
    feature_names=feature_names,
    output_path=os.path.join('train.cd')
)

Fit Model

In [ ]:

from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=5,
    learning_rate=0.1,
    # loss_function='CrossEntropy'
)
model.fit(
    X_train, y_train,
    verbose = True
)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())

Metrics

In [ ]:

model = CatBoostClassifier()
model.fit(X_train, y_train)
# Predicting result for training set and validation set
predict_val_rf2 = model.predict(X_validation)


# Model Performance

print("Accuracy : ", accuracy_score(y_validation, predict_val_rf2) *  100)
print("Recall : ", recall_score(y_validation, predict_val_rf2, pos_label = 'Attrited Customer') *  100)
print("Precision : ", precision_score(y_validation, predict_val_rf2, pos_label = 'Attrited Customer') *  100)
print(confusion_matrix(y_validation, predict_val_rf2))
print(classification_report(y_validation, predict_val_rf2))

In [ ]:
tp = sum([1 if actual_y == predicted_y and actual_y == 'Attrited Customer' else 0 for actual_y, predicted_y in zip(y_train,predict_val_rf2)])
tn = sum([1 if actual_y == predicted_y and actual_y == 'Existing Customer' else 0 for actual_y, predicted_y in zip(y_train,predict_val_rf2)])
fp = sum([1 if actual_y != predicted_y and actual_y == 'Attrited Customer' else 0 for actual_y, predicted_y in zip(y_train,predict_val_rf2)])
fn = sum([1 if actual_y != predicted_y and actual_y == 'Existing Customer' else 0 for actual_y, predicted_y in zip(y_train,predict_val_rf2)])

In [ ]:
print("Accuracy : ", accuracy_score(y_validation, predict_val_rf2) *  100)
